# Problem 3 - Computing the price of the 3Y7Y payer swaption

In [ ]:
# 3a) 3Y7Y swaption price
T_n, T_N = 3, 10
beta = 0.55
R_swaption, S_swaption = fid.swap_rate_from_zcb_prices(0,T_n,T_N,"annual",T_10Y_swap,p_10Y_swap)
print(f"ATMF 3Y7Y par swap rate: {R_swaption}")
N_swaption = len(K_swaption_offset)
K_swaption = K_swaption_offset/10000 + R_swaption*np.ones(N_swaption)
price_swaption_market = np.zeros([N_swaption])
for i in range(0,N_swaption):
    price_swaption_market[i] = fid.black_swaption_price(iv_swaption_market[i],T_n,K_swaption[i],S_swaption,R_swaption)
print(f"Payer swaption with strike closest to K=0.06. strike: {K_swaption[7]}, price: {price_swaption_market[7]*10000} bps")


ATMF 3Y7Y par swap rate: 0.05503331345783611
Payer swaption with strike closest to K=0.06. strike: 0.06003331345783611, price: 41.195444387893204 bps


In [ ]:
# 3b) Fitting the SABR model
param_0 = 0.025, 0.48,-0.25
result = minimize(fid.fit_sabr_no_beta_obj,param_0,method = 'nelder-mead',args = (beta,iv_swaption_market,K_swaption,T_n,R_swaption),options={'xatol': 1e-8,'disp': False})
sigma_0, upsilon, rho = result.x
print(f"Parameters from the SABR fit where beta: {beta} are: sigma_0: {sigma_0}, upsilon: {upsilon}, rho: {rho}, SSE: {result.fun}")
iv_fit, price_fit = np.zeros([N_swaption]), np.zeros([N_swaption])
for i in range(0,N_swaption):
    iv_fit[i] = fid.sigma_sabr(K_swaption[i],T_n,R_swaption,sigma_0,beta,upsilon,rho)
    price_fit[i] = fid.black_swaption_price(iv_fit[i],T_n,K_swaption[i],S_swaption,R_swaption,type_option = "call")


Parameters from the SABR fit where beta: 0.55 are: sigma_0: 0.017875223851695068, upsilon: 0.5840153631324319, rho: -0.3439552092629677, SSE: 2.798846154816731e-05


In [ ]:
# 3c) Price and risk management of a strangle
iv_payer_swaption = fid.sigma_sabr(K_swaption[8],T_n,R_swaption,sigma_0,beta,upsilon,rho)
price_payer_swaption_init = fid.black_swaption_price(iv_payer_swaption,T_n,K_swaption[8],S_swaption,R_swaption,type_option = "call")
iv_receiver_swaption = fid.sigma_sabr(K_swaption[4],T_n,R_swaption,sigma_0,beta,upsilon,rho)
price_receiver_swaption_init = fid.black_swaption_price(iv_receiver_swaption,T_n,K_swaption[4],S_swaption,R_swaption,type_option = "put")
print(f"Initially. Payer swaption: {price_payer_swaption_init*10000}, Receiver swaption: {price_receiver_swaption_init*10000}, Strangle: {price_payer_swaption_init*10000+price_receiver_swaption_init*10000}")
bps = 0.0001
iv_payer_swaption = fid.sigma_sabr(K_swaption[8],T_n,R_swaption + bps,sigma_0,beta,upsilon,rho)
price_payer_swaption = fid.black_swaption_price(iv_payer_swaption,T_n,K_swaption[8],S_swaption,R_swaption + bps,type_option = "call")
iv_receiver_swaption = fid.sigma_sabr(K_swaption[4],T_n,R_swaption + bps,sigma_0,beta,upsilon,rho)
price_receiver_swaption = fid.black_swaption_price(iv_receiver_swaption,T_n,K_swaption[4],S_swaption,R_swaption + bps,type_option = "put")
print(f"Par swap rate UP 1 bps. Payer swaption: {price_payer_swaption*10000}, Receiver swaption: {price_receiver_swaption*10000}, Strangle: {price_payer_swaption*10000+price_receiver_swaption*10000}, Difference: {10000*(price_payer_swaption+price_receiver_swaption-price_payer_swaption_init-price_receiver_swaption_init)}")
iv_payer_swaption = fid.sigma_sabr(K_swaption[8],T_n,R_swaption - bps,sigma_0,beta,upsilon,rho)
price_payer_swaption = fid.black_swaption_price(iv_payer_swaption,T_n,K_swaption[8],S_swaption,R_swaption - bps,type_option = "call")
iv_receiver_swaption = fid.sigma_sabr(K_swaption[4],T_n,R_swaption - bps,sigma_0,beta,upsilon,rho)
price_receiver_swaption = fid.black_swaption_price(iv_receiver_swaption,T_n,K_swaption[4],S_swaption,R_swaption - bps,type_option = "put")
print(f"Par swap rate DOWN 1 bps. Payer swaption: {price_payer_swaption*10000}, Receiver swaption: {price_receiver_swaption*10000}, Strangle: {price_payer_swaption*10000+price_receiver_swaption*10000}, Difference: {10000*(price_payer_swaption+price_receiver_swaption-price_payer_swaption_init-price_receiver_swaption_init)}")
iv_payer_swaption = fid.sigma_sabr(K_swaption[8],T_n,R_swaption,sigma_0 + 0.001,beta,upsilon,rho)
price_payer_swaption = fid.black_swaption_price(iv_payer_swaption,T_n,K_swaption[8],S_swaption,R_swaption,type_option = "call")
iv_receiver_swaption = fid.sigma_sabr(K_swaption[4],T_n,R_swaption,sigma_0 + 0.001,beta,upsilon,rho)
price_receiver_swaption = fid.black_swaption_price(iv_receiver_swaption,T_n,K_swaption[4],S_swaption,R_swaption,type_option = "put")
print(f"Volatility 0.001 UP. Payer swaption: {price_payer_swaption*10000}, Receiver swaption: {price_receiver_swaption*10000}, Strangle: {price_payer_swaption*10000+price_receiver_swaption*10000}, Difference: {10000*(price_payer_swaption+price_receiver_swaption-price_payer_swaption_init-price_receiver_swaption_init)}")
iv_payer_swaption = fid.sigma_sabr(K_swaption[8],T_n,R_swaption,sigma_0 - 0.001,beta,upsilon,rho)
price_payer_swaption = fid.black_swaption_price(iv_payer_swaption,T_n,K_swaption[8],S_swaption,R_swaption,type_option = "call")
iv_receiver_swaption = fid.sigma_sabr(K_swaption[4],T_n,R_swaption,sigma_0 - 0.001,beta,upsilon,rho)
price_receiver_swaption = fid.black_swaption_price(iv_receiver_swaption,T_n,K_swaption[4],S_swaption,R_swaption,type_option = "put")
print(f"Volatility 0.001 DOWN. Payer swaption: {price_payer_swaption*10000}, Receiver swaption: {price_receiver_swaption*10000}, Strangle: {price_payer_swaption*10000+price_receiver_swaption*10000}, Difference: {10000*(price_payer_swaption+price_receiver_swaption-price_payer_swaption_init-price_receiver_swaption_init)}")


Initially. Payer swaption: 16.053096667407942, Receiver swaption: 31.424116085775022, Strangle: 47.47721275318297
Par swap rate UP 1 bps. Payer swaption: 16.35234104159374, Receiver swaption: 31.13234003219374, Strangle: 47.48468107378748, Difference: 0.007468320604511747
Par swap rate DOWN 1 bps. Payer swaption: 15.76079054120206, Receiver swaption: 31.72019106310972, Strangle: 47.48098160431178, Difference: 0.003768851128812742
Volatility 0.001 UP. Payer swaption: 18.57246035677486, Receiver swaption: 35.03239454085507, Strangle: 53.604854897629934, Difference: 6.127642144446972
Volatility 0.001 DOWN. Payer swaption: 13.72821984857286, Receiver swaption: 27.959714097606827, Strangle: 41.68793394617968, Difference: -5.789278807003277


# Problem 5 - Short rate dynamics

In [ ]:
# problem 5
def mean_var(x0,a,b,sigma,y0,gamma,phi,T):
    N = len(T)
    mean = np.zeros(N)
    var = np.zeros(N)
    for n, t in enumerate(T):
        mean[n] = x0*np.exp(-gamma*T[n]) + y0*np.exp(-a*T[n]) + b/a*(1-np.exp(-a*T[n]))
        var[n] = phi**2/(2*gamma)*(1-np.exp(-2*gamma*T[n])) + sigma**2/(2*a)*(1-np.exp(-2*a*T[n]))
    return mean, var

# 5c) Simulating the factors driving the short rate and the short rate itself
x0, gamma, phi = 0, 32, 0.03
y0, a, b, sigma = 0.03, 0.5, 0.025, 0.015
M = 1000
T_st = 1
T_lt = 10
size_ci = 0.95
z = norm.ppf(size_ci + 0.5*(1-size_ci),0,1)
delta_st = T_st/M
t_simul_st = np.array([i*delta_st for i in range(0,M+1)])
X_st = fid.simul_vasicek(x0,gamma,0,phi,M,T_st)
Y_st = fid.simul_vasicek(y0,a,b,sigma,M,T_st)
r_st = X_st + Y_st
mean_st, var_st = mean_var(x0,a,b,sigma,y0,gamma,phi,t_simul_st)
lb_st, ub_st = mean_st - z*np.sqrt(var_st), mean_st + z*np.sqrt(var_st)
delta_lt = T_lt/M
t_simul_lt = np.array([i*delta_lt for i in range(0,M+1)])
X_lt = fid.simul_vasicek(x0,gamma,0,phi,M,T_lt)
Y_lt = fid.simul_vasicek(y0,a,b,sigma,M,T_lt)
r_lt = X_lt + Y_lt
mean_lt, var_lt = mean_var(x0,a,b,sigma,y0,gamma,phi,t_simul_lt)
lb_lt, ub_lt = mean_lt - z*np.sqrt(var_lt), mean_lt + z*np.sqrt(var_lt)
